# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv 01_materials/labs/.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\nares\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [11]:
import os
from glob import glob
import pandas as pd
import yfinance as yf
import os
import sys

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [12]:
# Write your code below.
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
).assign(
    hi_l0_range = lambda x: x['High'] - x['Low']
))


C:\Users\nares\AppData\Local\Temp\ipykernel_5692\466391307.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [13]:
#Write your code below.
pd_feat = dd_feat.compute()

In [14]:
pd_feat

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,positive_return,hi_l0_range
ticker,,,,,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.995823,35.126720,34.760205,34.825653,2039962,0.0,0.0,2013,NaN,NaN,0,0.366515
A,2013-12-03 00:00:00-05:00,34.635847,34.799472,34.400228,34.642391,3462706,0.0,0.0,2013,34.825653,-0.005262,0,0.399244
A,2013-12-04 00:00:00-05:00,34.583503,35.231455,34.504966,35.067829,3377288,0.0,0.0,2013,34.642391,0.012281,1,0.726489
A,2013-12-05 00:00:00-05:00,34.917283,35.211805,34.766750,35.015457,2530939,0.0,0.0,2013,35.067829,-0.001493,0,0.445055
A,2013-12-06 00:00:00-05:00,35.185637,35.951393,35.185637,35.885944,4268513,0.0,0.0,2013,35.015457,0.024860,1,0.765756
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024,128.165619,0.006026,1,1.752578
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024,128.937943,-0.008678,0,2.000117
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024,127.819061,0.011310,1,2.237759


In [15]:
pd_feat_rolling = pd_feat["returns"].rolling(10).mean()
pd_feat_rolling

ticker
A           NaN
A           NaN
A           NaN
A           NaN
A           NaN
         ...   
YUM    0.000828
YUM    0.000115
YUM    0.001145
YUM    0.000672
YUM    0.000370
Name: returns, Length: 118443, dtype: float64

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, it was not necessary to convert to pandas to calculate the moving average because dask is able to convert the dataframe to the moving average. Which means you could've saved time/disk space for calculating moving averages.

+ Would it have been better to do it in Dask? Why?

Yes, because dask is faster and uses less disk space. It uses parallel computing and lazy evaluation. In addition dask stores datasets in blocks and processes one block at a time when needed.


(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.